# Natural Language Processing

**THIS IS ONLY FOR 15-688 STUDENTS**

In this problem you will develop two techniques for analyzing free text documents: a bag of words approach based on a TFIDF matrix, and an n-gram language model.

You'll be applying your models to the text from the Federalist Papers.  The Federalist papers were a series of essay written in 1787 and 1788 by Alexander Hamilton, James Madison, and John Jay (they were published anonymously at the time), that promoted the ratification of the U.S. Constitution.  If you're curious, you can read more about them here: https://en.wikipedia.org/wiki/The_Federalist_Papers . They are a particularly interesting data set, because although the authorship of most of the essays has been long known since around the deaths of Hamilton and Madison, there was still some question about the authorship of certain articles into the 20th century.  You'll use document vectors and language models to do this analysis for yourself.

In [1]:
import collections # optional, but we found the collections.Counter object useful
import gzip
import re
import numpy as np
import scipy.sparse as sp
from testing.testing import test 

## The dataset

You'll use a copy of the Federalist Papers downloaded from Project Guttenberg, available here: http://www.gutenberg.org/ebooks/18 .  Specifically, the "pg18.txt.gz" file included with the homework is the raw text downloaded from Project Guttenberg.  To ensure that everyone starts with the exact same corpus, we are providing you the code to load and tokenize this document, as given below.

In [2]:
def load_federalist_corpus(filename):
    """ Load the federalist papers as a tokenized list of strings"""
    with gzip.open(filename, "rt", encoding="utf-8") as f:
        data = f.read()
    papers = data.split("FEDERALIST")
    
    # all start with "To the people of the State of New York:" (sometimes . instead of :)
    # all end with PUBLIUS (or no end at all)
    locations = [(i,[-1] + [m.end()+1 for m in re.finditer(r"of the State of New York", p)],
                 [-1] + [m.start() for m in re.finditer(r"PUBLIUS", p)]) for i,p in enumerate(papers)]
    papers_content = [papers[i][max(loc[1]):max(loc[2])] for i,loc in enumerate(locations)]

    # discard entries that are not actually a paper
    papers_content = [p for p in papers_content if len(p) > 0]

    # replace all whitespace with a single space
    papers_content = [re.sub(r"\s+", " ", p).lower() for p in papers_content]

    # add spaces before all punctuation, so they are separate tokens
    punctuation = set(re.findall(r"[^\w\s]+", " ".join(papers_content))) - {"-","'"}
    for c in punctuation:
        papers_content = [p.replace(c, " "+c+" ") for p in papers_content]
    papers_content = [re.sub(r"\s+", " ", p).lower().strip() for p in papers_content]
    
    authors = [tuple(re.findall("MADISON|JAY|HAMILTON", a)) for a in papers]
    authors = [a for a in authors if len(a) > 0]
    
    numbers = [re.search(r"No\. \d+", p).group(0) for p in papers if re.search(r"No\. \d+", p)]
    
    return papers_content, authors, numbers
    
    

You're welcome to dig through the code here if you're curious, but it's more important that you look at the objects that the function returns.  The `PAPERS` object is a list of strings, each one containing the full content of one of the Federalist Papers.  All tokens (words) in the text are separated by a single space (this includes some puncutation tokens, which have been modified to include spaces both before and after the punctuation. The `AUTHORS` object is a list of lists, which each list contains the author (or potential authors) of a given paper.  Finally the `NUMBERS` list just contains the number of each Federalist paper.  You won't need to use this last one, but you may be curious to compare the results of your textual analysis to the opinion of historians.

## Q1: Bag of words, and TFIDF

In this portion of the question, you'll use a bag of words model to describe the corpus, and write routines to build a TFIDF matrix and a cosine similarity function.  Specifically, you should first implement the TFIDF function below.  This should return a _sparse_ TFIDF matrix (as for the Graph Library assignment, make sure to directly create a sparse matrix using `scipy.sparse.coo_matrix()` rather than create a dense matrix and then convert it to be sparse).

You should create the tfidf vector using numpy matrix operations and not use an existing implementation.

Important: make sure you do _not_ include the empty token `""` as one of your terms. 

In [3]:
TEST_DATA = ["the goal of this lecture is to explain the basics of free text processing",
             "the bag of words model is one such approach",
             "text processing via bag of words"]

def tfidf_test(tfidf_impl):
    X_tfidf, words = tfidf_impl(TEST_DATA)
    test.equal(X_tfidf.shape, (3, 19))
    test.equal(set(words), {'one', 'bag', 'goal', 'explain', 'approach', 'to', 'processing', 'this', 'the', 'model', 'basics', 'free', 'words', 'such', 'is', 'text', 'lecture', 'via', 'of'})
    test.equal(X_tfidf[0, words.index('explain')], 1.0986122886681098)
    
@test
def tfidf(docs):
    """
    Create TFIDF matrix.  This function creates a TFIDF matrix from the
    docs input.

    Args:
        docs: list of strings, where each string represents a space-separated
              document
    
    Returns: tuple: (tfidf, all_words)
        tfidf: sparse matrix (in any scipy sparse format) of size (# docs) x
               (# total unique words), where i,j entry is TFIDF score for 
               document i and term j
        all_words: list of strings, where the ith element indicates the word
                   that corresponds to the ith column in the TFIDF matrix
    """
    ##### do not include "" as one of the terms
    
    ## 1. create term frequency matrix
    # construct list for all text
    document_words = [doc.split() for doc in docs]
    vocab = sorted(set(sum(document_words, [])))
    vocab_dict = {k:i for i,k in enumerate(vocab)}
    for i,k in enumerate(vocab):
        if k!= "":
            vocab_dict[k] = i
        
#     print(vocab_dict)
    dic = vocab_dict
    # 
    col_idx = []
    row_idx = []
    val = []
    # construct sparse matrix
    for i in range(len(docs)):
        # split the sample
        sample = docs[i].split(' ')
        temp_col = []
        temp_row = []
        temp_val = []
        for j in sample:
            index = dic[j]
            if index not in temp_col: # new element
                temp_col.append(index)
                temp_row.append(i)
                temp_val.append(1)
            else: # old element
                idx = temp_col.index(index)
                temp_val[idx] = temp_val[idx] + 1
                
        col_idx = col_idx + temp_col
        row_idx = row_idx + temp_row
        val = val + temp_val

    X_tf = sp.coo_matrix((val,(row_idx,col_idx)), shape=(len(docs),len(dic.keys())))
#     # print(X_tf.todense())
    
    ## 2. create inverse document frequency
    # temp 
    temp = [0] * len(dic)
    for i in range(len(dic)):
        count = col_idx.count(i)
        temp[i] = np.log((len(docs) / float(count)))
    # print(temp)
    
    ## 3. create TFIDF matrix
    for i in range(len(col_idx)):
        val[i] = val[i] * temp[col_idx[i]]
    X_tfidf = sp.csr_matrix((val,(row_idx,col_idx)), shape=(len(docs),len(dic.keys())))
    # print(X_tfidf.todense())
    return X_tfidf, vocab

### TESTING tfidf: PASSED 3/3
###



Our version results the following result (just showing the type, size, and # of non-zero elements):

    <86x8686 sparse matrix of type '<type 'numpy.float64'>'
        with 57607 stored elements in Compressed Sparse Row format>

Next, implement the following simple function that takes the X_tfidf matrix (though it could also take simple term frequency matrices, etc), and compute a matrix of all pair-wise cosine similarities.

In [4]:
def cosine_similarity_test(cosine_similarity_impl):
    X_tfidf, words = tfidf(TEST_DATA)
    M = cosine_similarity_impl(X_tfidf)
    test.true(np.allclose(M, 
        np.matrix([[1., 0.06796739, 0.07771876], [0.06796739, 1., 0.10281225], [0.07771876, 0.10281225, 1.]])))

@test
def cosine_similarity(X):
    """
    Return a matrix of cosine similarities.
    
    Args:
        X: sparse matrix of TFIDF scores or term frequencies
    
    Returns:
        M: dense numpy array of all pairwise cosine similarities.  That is, the 
           entry M[i,j], should correspond to the cosine similarity between the 
           ith and jth rows of X.
    """
    # calculate the norm
    data = np.array(X.data,dtype=float) 
    indices = X.indices
    indptr = X.indptr
    temp = X.tocoo()
    row_idx = temp.row
    col_idx = temp.col
    row_set = set(row_idx)
    norm_data = []
    norm_list = []
    for j in row_set:
        temp = []
        for i in range(len(row_idx)):
            if row_idx[i] == j:
                temp.append(data[i])
        t = np.linalg.norm(temp)
        norm_list.append(t)
        temp = temp / t
        norm_data = norm_data + temp.tolist()
    X_norm = sp.csr_matrix((norm_data,(row_idx,col_idx)),X.shape)
    
#     print(X_norm.todense())
    M = X_norm @ X_norm.T
    return M.todense()

### TESTING cosine_similarity: PASSED 1/1
###



Finally, use this model to analyze potential authorship of the unknown Federalist Papers.  Specifically, compute the average cosine similarity between all the _known_ Hamilton papers and all the _unknown_ papers (and similarly between known Madison and unknown, and Jay and unknown).  Fill out the following function to compute and return these averages.

In [5]:
def author_cosine_similarity_test(author_cosine_similarity_impl):
    papers, authors, numbers = load_federalist_corpus("pg18.txt.gz")
    hamilton_mcs, madison_mcs, jay_mcs = author_cosine_similarity_impl(papers, authors)
    test.equal(np.round(jay_mcs, 4), 0.0649)

    

@test
def author_cosine_similarity(docs, authors):
    """
    Return a tuple of average cosine similarities between all the known papers for a given author
    and all the unknown papers.
    
    Args:
        docs: list of strings, where each string represents a space-separated
              document
        authors: list of lists, which each list contains the author (or potential authors) of a given document
    
    Returns: tuple: (hamilton_mcs, madison_mcs, jay_mcs)
        hamilton_mcs: Average cosine similarity between all the known Hamilton papers and all the unknown papers.
        madison_mcs: Average cosine similarity between all the known Madison papers and all the unknown papers.
        jay_mcs: Average cosine similarity between all the known Jay papers and all the unknown papers.
    """
    total,_ = tfidf(docs)
    M = cosine_similarity(total)
    
    
    l_ham = []
    l_mad = []
    l_jay = []
    
    ## create lists
    for i in range(len(authors)):
        for j in range(len(authors)):
            if (len(authors[i]) == 1) and (len(authors[j]) == 2):
                if (authors[i][0] == 'HAMILTON'):
                    l_ham.append(M[i,j])
                elif authors[i][0] == 'JAY':
                    l_jay.append(M[i,j])
                elif authors[i][0] == 'MADISON':
                    l_mad.append(M[i,j])
    hamilton_mcs = sum(l_ham) / len(l_ham)
    madison_mcs = sum(l_mad) / len(l_mad)
    jay_mcs = sum(l_jay) / len(l_jay)
    return hamilton_mcs, madison_mcs, jay_mcs


### TESTING author_cosine_similarity: PASSED 1/1
###



## Q2: N-gram language models

In this question, you will implement an n-gram model to be able to model the language used in the Federalist Papers in a more structured manner than the simple bag of words approach.  You will fill in the following class:

In [6]:
def LanguageModel_test(LanguageModel_impl):
    papers, authors, numbers = load_federalist_corpus("pg18.txt.gz")
    l_hamilton = LanguageModel_impl(papers, 3)
    test.equal(l_hamilton.counts["privilege of"], {'being': 1, 'proposing': 1, 'residence': 1, 'originating': 1, 'paying': 1, 'the': 1})
    test.equal(l_hamilton.count_sums["privilege of"], 6)
    test.equal(np.round(l_hamilton.perplexity(papers[0]), 4), 15.1144)

    # Simple Test
    l_mini = LanguageModel_impl(["a a a a a", "a b a b"], 2)
    # Counts
    test.equal(l_mini.counts, {'a': {'a': 4, 'b': 2}, 'b': {'a': 1}})
    test.equal(l_mini.count_sums, {'a': 6, 'b': 1})
    # Simple perplexity checks:
    test.equal(round(l_mini.perplexity("a a"), 3), 1.500)
    test.equal(round(l_mini.perplexity("a a a"), 3), 1.500)
    test.equal(round(l_mini.perplexity("a b"), 3), 3.000)
    test.equal(round(l_mini.perplexity("a a b"), 3), 2.121)
    test.equal(round(l_mini.perplexity("b a"), 3), 1.001)
    # Check perplexity when the predecessor is present but not the successor
    test.equal(round(l_mini.perplexity("a c"), 3), 6003.0)
    # Check perplexity when the predecessor is not present
    test.equal(round(l_mini.perplexity("c a"), 3), 3.000)
    
    # Low-likelihood string:
    test.equal(np.round(l_hamilton.perplexity("running with scissors"), 4), 8687.0)
    # High-likelihood string:
    test.equal(np.round(l_hamilton.perplexity("continue to be"), 4), 2.9547)

@test
class LanguageModel:
    def __init__(self, docs, n):
        """
        Initialize an n-gram language model.
        
        Args:
            docs: list of strings, where each string represents a space-separated
                  document
            n: integer, degree of n-gram model
        """
        # self.counts: two-level dictionary, first: n-1, second: nth
        self.counts = {}
        # self.count_sums
        self.count_sums = {}
        # self.dictionary
        self.dictionary = set()
        self.n = n
        
        # create a list of input
        docs_list = []

        # future dic
        unique = []
        # dic
        counts = {}
        counts_sums = {}
        length = 0
        # loop the dic
        for j in docs:
            docs_list = j.split()
            length = length + len(docs_list) - n + 1
            for i in range(n-1,len(docs_list)):
                window = docs_list[i-n+1:i]
                key = ' '.join(window)
    #             # print(key)
                # first key appears first
                if key not in counts.keys():

                    val = {}
                    val[docs_list[i]] = 1
                    counts[key] = val
                    # print(counts)
                else:
                    # if val is the first time appear
                    if docs_list[i] not in counts[key]:
                        counts[key][docs_list[i]] = 1
                        # print(counts)
                    # if val is not the first time appear
                    else:
                        counts[key][docs_list[i]] = counts[key][docs_list[i]] + 1
                        # print(counts)
        self.len = length
        # create counts_sums
        for k,v in counts.items():
            counts_sums[k] = sum(v.values())
        
        # create unique:
        
        for i in docs:
            docs_list = docs_list + i.split()
        unique = set(docs_list)
        self.len = len(docs_list)
        self.counts = counts
        self.count_sums = counts_sums
        self.dictionary = unique
        
    
    def perplexity(self, text, alpha=1e-3):
        """
        Evaluate perplexity of model on some text.
        
        Args:
            text: string containing space-separated words, on which to compute
            alpha: constant to use in Laplace smoothing
            
        Note: for the purposes of smoothing, the dictionary size (i.e, the D term)
        should be equal to the total number of unique words used to build the model
        _and_ in the input text to this function.
            
        Returns: perplexity
            perplexity: floating point value, perplexity of the text as evaluted
                        under the model.
        """
        # log should be added
        # use laplace smooth
        # evaluate the perplexity on some words
        import math
        # cal D
        temp1 = list(self.dictionary)
        docs_list = []

        docs_list = text.split()

        temp2 = list(set(docs_list))
        D = len(set(temp1 + temp2))

        N = len(docs_list) - self.n + 1
             
        counts = self.counts
        count_sums = self.count_sums
        
        log_sum = 0
        # itera and create the key
        for i in range(self.n-1, len(docs_list)):
            window = docs_list[i-self.n+1:i]
            key = ' '.join(window)
            try:
                temp1 = counts[key][docs_list[i]]
            except:
                temp1 = 0
            try:
                temp2 = count_sums[key]
            except:
                temp2 = 0
            p = (temp1 + alpha) / (temp2 + alpha * D)
            log_sum = log_sum + math.log(p,2)
        # 
        temp = (-log_sum) / N
        perplexity = 2 ** temp
        return perplexity
            

### TESTING LanguageModel: PASSED 14/14
###



### Part A: Initializing the language model

First, implement the `__init__()` function in the `LanguageModel` class.  You can store the information any way you want, but we did this by building a two-level dictionary (in fact, we used the `collections.defaultdict` class, but this only make a few things a little bit shorter ... you are free to use it or not) `self.counts`, where the first key refers to the previous $n-1$ tokens, and the second key refers to the $n$th token, and the value simply stores the count of the number of times this combination was seen.  For ease of use in later function, we also created a `self.count_sums`, which contained the number of total times each $n-1$ combination was seen. We also build a `self.dictionary` variable, which is just a `set` object containing all the unique words in across the entire set of input document.

### Part B: Computing perplexity

Next, implement the `perplexity()` function, which takes a text sample and computes the perplexity of this sample under the model.  Use the formula for perplexity from the class nodes (which is actually not exact, since it only so, being careful to not multiply togther probabilites that get too small (hint: instead of taking the log of a large product, take the sum of the logs).

You'll want to be careful about dictionary sizes when it comes to the Laplace smoothing term: make sure your dictionary size $D$ is equal to the total number of unique words that occur in either the original data used to build the language model _or_ in the text we are evaluating the perplexity of (so the size of the union of the two).

Using this model, evaluate the mean of the perplexity of the unknown Federalist papers for the language models from each of the three authors (again, using `n=3` and the default of `alpha=1e-3`). Fill in the following function to calculate and return the mean perplexities.

In [9]:
def mean_perplexity_test(mean_perplexity_impl):
    papers, authors, numbers = load_federalist_corpus("pg18.txt.gz")
    perp_hamilton, perp_madison, perp_jay = mean_perplexity_impl(papers, authors)
    test.equal(np.round(perp_hamilton, 4), 1941.3852)

@test
def mean_perplexity(docs, authors):
    """
    Evaluate the mean of the perplexity of the unknown Federalist papers for the language models
    from each of the three authors (again, using n=3 and alpha=1e-3)

    Args:
        docs: list of strings, where each string represents a space-separated document
        authors: list of lists, which each list contains the author (or potential authors) of a given document

    Returns: tuple: (perp_hamilton, perp_madison, perp_jay)
        perp_hamilton: floating point value, mean perplexity of the unknown Federalist papers for the language 
                       models from Hamilton
        perp_madison: floating point value, mean perplexity of the unknown Federalist papers for the language 
                       models from Madison
        perp_jay: floating point value, mean perplexity of the unknown Federalist papers for the language 
                       models from Jay
    """
    # language model for hamilton, etc.
    train_hamilton = []
    train_madison = []
    train_jay = []
    test_unknown = []
    
    # split data
    for i in range(len(authors)):
        if len(authors[i]) == 1:
            if authors[i][0] == 'HAMILTON':
                train_hamilton.append(docs[i])
            elif authors[i][0] == 'JAY':
                train_jay.append(docs[i])
            elif authors[i][0] == 'MADISON':
                train_madison.append(docs[i])
        else:
            test_unknown.append(docs[i])
    
    # train models
    hamilton = LanguageModel(train_hamilton,3)
    h = []
    madison = LanguageModel(train_madison,3)
    m = []
    jay = LanguageModel(train_jay,3)
    j = []
    for i in test_unknown:
        h.append(hamilton.perplexity(i))
        m.append(madison.perplexity(i))
        j.append(jay.perplexity(i))
    return sum(h) / len(h), sum(m) / len(m), sum(j) / len(j)
                

1941.3851677346731
1796.9137036593322
2027.895787270102
### TESTING mean_perplexity: PASSED 1/1
###

